In [1]:
import pandas as pd
import numpy as np
import re,json,requests

api key: 1e3b5c39

In [2]:
ratings_data = pd.read_csv("data/ml-latest-small/ratings.csv", delimiter=",", encoding="ISO-8859-1")
movie_data = pd.read_csv("data/ml-latest-small/movies.csv", delimiter=",", encoding="ISO-8859-1",index_col=0)
movie_data["genres"] = movie_data["genres"].str.replace("|",",")
movie_data

,title,genres
movieId,,
1,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy"
2,Jumanji (1995),"Adventure,Children,Fantasy"
3,Grumpier Old Men (1995),"Comedy,Romance"
4,Waiting to Exhale (1995),"Comedy,Drama,Romance"
5,Father of the Bride Part II (1995),Comedy
6,Heat (1995),"Action,Crime,Thriller"
7,Sabrina (1995),"Comedy,Romance"
8,Tom and Huck (1995),"Adventure,Children"
9,Sudden Death (1995),Action


In [3]:
def rottenScore(l):
    rotten = np.nan
    if type(l) is not float:
        for d in l:
            if "Rotten Tomatoes" in d.values():
                rotten = float(re.match(r'(.+)\%',d["Value"]).group(1))*0.05
    return rotten

In [4]:
def cleanVotes(v):
    if type(v) is not float:
        return int(v.replace(',',''))
    else:
        return v

In [10]:
movies = pd.DataFrame(columns=['Actors', 'Awards', 'BoxOffice', 'Country', 'DVD', 'Director', 'Genre',
       'Language', 'Metascore', 'Plot', 'Poster', 'Production', 'Rated',
       'Ratings', 'Released', 'Response', 'Runtime', 'Title', 'Type',
       'Website', 'Writer', 'Year', 'imdbID', 'imdbRating', 'imdbVotes', 'movieId'])
parser = re.compile(r'([a-zA-Z:Ã©¡³º¨±¶€¤\' 0-9-]+).+\((\d*)\)$')
unicodes = {"Ã¡":"á","Ã©":"é","Ã":"í","Ã³":"ó","Ãº":"ú","Ã¨":"è","Ã±":"ñ","Ã¶":"ö","Ã€":"À","Ã¤":"ä"}
replaces = re.compile('|'.join(unicodes.keys()))
indexes = movie_data.index.values
r = range(5250,5500)
for i in r:
    try:
        mov = movie_data.loc[i,"title"]
        raw_title = parser.match(mov).group(1).strip()
        raw_title = replaces.sub(lambda x: unicodes[x.group()], raw_title)
        title = raw_title.replace(" ","%20").replace("&","%26")
        raw_year = parser.match(mov).group(2).strip().replace(" ","%20")
        year = raw_year.replace(" ","%20")
        if len(year) > 0:
            year = "&y=" + year
        api_call = "http://www.omdbapi.com/?apikey=1e3b5c39&t=" + title + year
        print(api_call)
        r = requests.get(api_call)
        if r.status_code == 200:
            r = json.loads(r.text)
            if r["Response"] == "True":
                norm = pd.io.json.json_normalize(r)
                norm = norm.assign(movieId = str(indexes[i-1]))
                movies = movies.append(norm)
            else:
                movies = movies.append(pd.Series({'Actors' : np.nan, 'Awards': np.nan, 'BoxOffice': np.nan, 
                                                     'Country': np.nan, 'DVD': np.nan, 'Director': np.nan, 
                                                     'Genre' : movie_data.loc[i,"genres"], 'Language': np.nan, 'Metascore': np.nan, 
                                                     'Plot': np.nan, 'Poster': np.nan, 'Production': np.nan, 'Rated': np.nan,
               'Ratings': np.nan, 'Released': np.nan, 'Response': np.nan, 'Runtime': np.nan, 'Title': raw_title, 'Type': np.nan,
               'Website': np.nan, 'Writer': np.nan, 'Year': raw_year, 'imdbID': np.nan, 'imdbRating': np.nan, 
                                                     'imdbVotes': np.nan, 'movieId' : str(indexes[i-1])}),ignore_index=True)
    except:
        pass
movies.set_index('movieId', inplace=True)
movies.rename(columns={c:c.lower() for c in movies.columns},inplace=True)
movies[movies[:]=="N/A"] = np.nan
movies.drop(["boxoffice","response"],axis=1,inplace=True)
try:
    movies["actors"] = movies["actors"].str.split(",")
except:
    pass
try:
    movies["country"] = movies["country"].str.split(",")
except:
    pass
try:
    movies["director"] = movies["director"].str.split(",")
except:
    pass
try:
    movies["language"] = movies["language"].str.split(",")
except:
    pass
try:
    movies["genre"] = movies["genre"].str.split(",")
except:
    pass
try:
    movies["writer"] = movies["writer"].str.split(",")
except:
    pass
try:
    movies["runtime"] = movies["runtime"].apply(lambda x : int(re.match(r'(\d+).*min', x).group(1)))
except:
    pass
movies["dvd"] = movies["dvd"].apply(lambda x:pd.to_datetime(x))
movies["released"] = movies["released"].apply(lambda x:pd.to_datetime(x))
movies["metascore"] = movies["metascore"].apply(lambda x:float(x)*0.05)
movies["imdbrating"] = movies["imdbrating"].apply(lambda x:float(x)*0.5)
movies["imdbvotes"] = movies["imdbvotes"].apply(cleanVotes)
movies["ratings"] = movies["ratings"].apply(rottenScore)
movies.rename(columns={"ratings":"rottenscore"},inplace=True)
movies.to_csv("pandasdata/movie_data22", encoding="utf-8")
movies

http://www.omdbapi.com/?apikey=1e3b5c39&t=Stir%20Crazy&y=1980
http://www.omdbapi.com/?apikey=1e3b5c39&t=Blade%20II&y=2002
http://www.omdbapi.com/?apikey=1e3b5c39&t=Sorority%20Boys&y=2002
http://www.omdbapi.com/?apikey=1e3b5c39&t=Stolen%20Summer&y=2002
http://www.omdbapi.com/?apikey=1e3b5c39&t=George%20Washington&y=2000
http://www.omdbapi.com/?apikey=1e3b5c39&t=Clockstoppers&y=2002
http://www.omdbapi.com/?apikey=1e3b5c39&t=Death%20to%20Smoochy&y=2002
http://www.omdbapi.com/?apikey=1e3b5c39&t=Panic%20Room&y=2002
http://www.omdbapi.com/?apikey=1e3b5c39&t=Rookie&y=2002
http://www.omdbapi.com/?apikey=1e3b5c39&t=Piano%20Teacher&y=2001
http://www.omdbapi.com/?apikey=1e3b5c39&t=30%20Years%20to%20Life&y=2001
http://www.omdbapi.com/?apikey=1e3b5c39&t=Boxcar%20Bertha&y=1972
http://www.omdbapi.com/?apikey=1e3b5c39&t=Crimes%20of%20Passion&y=1984
http://www.omdbapi.com/?apikey=1e3b5c39&t=Evil%20That%20Men%20Do&y=1984
http://www.omdbapi.com/?apikey=1e3b5c39&t=Fraternity%20Vacation&y=1985
http://www.o

,actors,awards,country,dvd,director,genre,language,metascore,plot,poster,...,released,runtime,title,type,website,writer,year,imdbid,imdbrating,imdbvotes
movieId,,,,,,,,,,,,,,,,,,,,,
7831,"[Gene Wilder, Richard Pryor, Georg Stanford ...",1 nomination.,[USA],2001-08-28,[Sidney Poitier],"[Comedy, Crime]",[English],2.80,"Set up and wrongfully accused, two best friend...",https://images-na.ssl-images-amazon.com/images...,...,1980-12-12,111 min,Stir Crazy,movie,NaN,[Bruce Jay Friedman],1980,tt0081562,3.40,20611.0
7835,"[Wesley Snipes, Kris Kristofferson, Ron Perl...",6 wins & 10 nominations.,"[Germany, USA]",2002-09-03,[Guillermo del Toro],"[Action, Horror, Sci-Fi]","[English, Romanian, Czech]",2.60,Blade forms an uneasy alliance with the vampir...,https://images-na.ssl-images-amazon.com/images...,...,2002-03-22,117 min,Blade II,movie,http://www.blade2.com,"[Marv Wolfman (character), Gene Colan (charac...",2002,tt0187738,3.35,174720.0
7836,"[Barry Watson, Michael Rosenbaum, Harland Wi...",1 win & 1 nomination.,[USA],2002-10-15,[Wallace Wolodarsky],[Comedy],"[English, Indonesian]",1.25,Three rowdy college students are kicked out of...,https://images-na.ssl-images-amazon.com/images...,...,2002-03-22,93 min,Sorority Boys,movie,http://bventertainment.go.com/movies/sororityboys,"[Joe Jarvis, Greg Coolidge]",2002,tt0279781,2.70,12642.0
7840,"[Aidan Quinn, Bonnie Hunt, Kevin Pollak, Ed...",NaN,[USA],2002-09-24,[Pete Jones],[Drama],[English],1.80,A Catholic boy tries to convert a terminally i...,https://images-na.ssl-images-amazon.com/images...,...,2003-07-25,91 min,Stolen Summer,movie,http://projectgreenlight.liveplanet.com,[Pete Jones],2002,tt0286162,3.25,2495.0
7842,"[Candace Evanofski, Donald Holden, Damian Je...",9 wins & 15 nominations.,[USA],2002-03-12,[David Gordon Green],[Drama],[English],4.10,"A group of children, in a depressed small town...",https://images-na.ssl-images-amazon.com/images...,...,2001-09-28,89 min,George Washington,movie,NaN,[David Gordon Green],2000,tt0262432,3.70,6757.0
7878,"[Jesse Bradford, French Stewart, Paula Garcé...",NaN,[USA],2002-08-13,[Jonathan Frakes],"[Action, Adventure, Comedy]",[English],2.00,A teenager accidentally activates a machine th...,https://images-na.ssl-images-amazon.com/images...,...,2002-03-29,94 min,Clockstoppers,movie,http://www.clockstoppers.com,"[Rob Hedden (story), Andy Hedden (story), J....",2002,tt0157472,2.60,12095.0
7881,"[Robin Williams, Edward Norton, Catherine Ke...",2 nominations.,"[UK, Germany, USA]",2002-09-17,[Danny DeVito],"[Comedy, Crime, Drama]",[English],1.90,"A kids show host, Rainbow Randolph, is fired i...",https://images-na.ssl-images-amazon.com/images...,...,2002-03-29,109 min,Death to Smoochy,movie,http://deathtosmoochymovie.warnerbros.com,[Adam Resnick],2002,tt0266452,3.20,36123.0
7882,"[Jodie Foster, Kristen Stewart, Forest Whita...",1 win & 9 nominations.,[USA],2002-09-17,[David Fincher],"[Crime, Drama, Thriller]",[English],3.25,A divorced woman and her diabetic daughter tak...,https://images-na.ssl-images-amazon.com/images...,...,2002-03-29,112 min,Panic Room,movie,http://www.spe.sony.com/movies/panicroom,[David Koepp],2002,tt0258000,3.40,224377.0
7883,"[Dennis Quaid, Rachel Griffiths, Jay Hernand...",5 wins & 4 nominations.,[USA],2002-08-27,[John Lee Hancock],"[Drama, Family, Sport]",[English],3.60,A Texas baseball coach makes the major league ...,https://images-na.ssl-images-amazon.com/images...,...,2002-03-29,127 min,The Rookie,movie,http://disney.go.com/disneypictures/rookie/ind...,[Mike Rich],2002,tt0265662,3.50,28383.0
